# Clase Práctica 10

# CNN - Redes Neuronales Convolucionales

# Modelos pre-entrenados con ajuste fino y transferencia de aprendizaje


Los términos aprendizaje de transferencia y ajuste fino se refieren a dos conceptos que son muy similares en muchos aspectos, y los dos términos se utilizan ampliamente de manera casi intercambiable. Los dos términos no implican el mismo objetivo o motivación, pero aún se refieren a un concepto similar: 

1) la sintonía precisa o ajuste fino (fine tuning) es tomar un modelo de aprendizaje automático que ya ha aprendido algo antes (es decir, haber sido entrenado en algunos datos) y luego entrenarlo (es decir, entrenarlo un poco más, posiblemente con datos diferentes). Eso es todo lo que significa afinar. 

2) la transferencia de aprendizaje (transfer learning) significa aplicar el conocimiento que posee un modelo de Deep Learning (representado por sus parámetros aprendidos) a una nueva tarea (pero de alguna manera relacionada). 

La razón principal por la que las personas utilizan el término ajuste fino es simplemente para indicar que el modelo no se está entrenando desde cero, y que ya se ha entrenado anteriormente en algunos datos (no necesariamente en convergencia). Solo es una forma conveniente de expresar que no estás entrenando desde cero. 

# Ajuste Fino -  Fine-tuning 

El motivo principal para usar fine-tuning es el ahorro en recursos de computación y tiempo, pues evitamos la mayor parte del entrenamiento.

Si disponemos de acceso a una red pre-entrenada que resuelva un problema similar al que queremos resolver, podemos usar esa red como punto de partida. Simplemente la adaptamos a nuestro problema y continuamos entrenando con nuestros datos. La red ya ha aprendido a extraer las características universales necesarias, por lo que la mayor parte del trabajo ya está hecho.

Si disponemos de una red pre-entrenada para un problema similar y disponemos de muchos datos de entrenamiento para nuestro problema concreto, estamos ante el caso ideal. Simplemente usaremos la red pre-entrenada como punto de partida y continuaremos el entrenamiento con nuestros datos.

Pero, si tenemos una red pre-entrenada para un problema similar pero tenemos pocos datos de entrenamiento debemos proceder diferente. Entrenar una red con pocos datos nos llevará fácilmente a una situación de overfitting, donde la red aprende a clasificar los datos de entrenamiento pero no los de test (las redes neuronales suelen necesitar muchos datos para poder aprender a generalizar). En este caso entrenaremos solo las últimas capas de la red, congelando el resto. La red ya está entrenada para un problema similar, por lo que la usaremos principalmente como extractor de características.

Por otra parte, si disponemos de muchos datos de entrenamiento, la mejor solución suele ser entrenar la red desde cero, aunque esto requiere de mucho tiempo de entrenamiento.

Pero si disponemos de pocos datos de entrenamiento la opción anterior no suele ser viable debido al overfitting, a menos que podamos usar con éxito alguna técnica de data augmentation. Ante esta situación, una opción suele ser congelar únicamente las primeras capas de la red. Estas capas han aprendido a extraer características universales por lo que pueden ser aprovechadas.


#  Importando Datos

Como de costumbre, debemos cargar las librerias necesarias para poder realizar las tareas de clasificación. En esta parte del código, se muestra como cargar las imagenes de una base de datos. Las imágenes vienen en diferentes directorios (carpetas) y usamos el nombre de cada directorio como nombre de la clase. El código asigna un identificador numérico a cada clase, es decir si hay 5 directorios (carpetas), nuestro código asigna 5 clases (clase: 0, 1, 2, 3, 4). 

In [ ]:
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input
from sklearn.model_selection import train_test_split
import numpy as np
import os
import glob
import matplotlib.pyplot as plt
from keras.utils import np_utils
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D


# parámetros
num_classes = 5
img_rows, img_cols = 100, 100
input_shape = (img_rows, img_cols,3)

# función para cargar las imágenes 

def load_data(path, formato):
    class_names={}
    class_id=0
    Xx = []
    Yy = []
    for d in glob.glob(os.path.join(path, '*')):
        clname = os.path.basename(d)
        for f in glob.glob(os.path.join(d, formato)): 
            if not clname in class_names:
                class_names[clname]=class_id 
                class_id += 1
            img = image.load_img(f, target_size=(img_rows, img_cols))
            npi = image.img_to_array(img)       
            #npi = preprocess_input(npi)
            Xx.append(npi)
            Yy.append(class_names[clname])
    return np.array(Xx), np.array(Yy), class_names


#  Cargando la base de datos

In [ ]:
# cargar las imágenes, etiquetas y nombres de las clases
Xx, Yy, class_names = load_data('flower_photos', '*.jpg')
#Xx, Yy, class_names = load_data('profesores', '*.png')

num_classes = len(class_names)

print("Las siguientes imágenes fueron exportadas (ejemplos, fila, col, prof) :")
print(Xx.shape)
print("Las siguientes etiquetas y clases fueron exportadas:")
print(Yy.shape, len(class_names))

# Plot

In [ ]:
def normalizar_imagen(imagen):
    maxi=(np.max(imagen))
    mini=(np.min(imagen))
    i2 = (imagen-mini)/(maxi-mini)
    plt.imshow(i2)

normalizar_imagen(Xx[0,:,:,:])

#  Dividiendo las imágenes

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Xx, Yy, test_size=0.1)

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.3)

x_train = x_train.astype('float32')/ 255
x_val = x_val.astype('float32') / 255 
x_test = x_test.astype('float32') /255

print('x_train shape:', x_train.shape)
print('x_val shape:', x_val.shape)
print('x_test shape:', x_test.shape)
print(x_train.shape[0], 'train samples')
print(x_val.shape[0], 'val samples')
print(x_test.shape[0], 'test samples')
print(y_train.shape[0], 'train labels')
print(y_val.shape[0], 'val labels')
print(y_test.shape[0], 'test labels')

YY_train=y_train # respaldar variables originales sin hot encoding
YY_val=y_val
YY_test=y_test
# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_val = np_utils.to_categorical(y_val, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

normalizar_imagen(x_train[0])

In [ ]:
i = x_train[0]
print("Valores mínimos y máximos de la imagen")
maxi=(np.max(i))
mini=(np.min(i))
print(mini,maxi )

#  Redes pre-entrenadas en Keras

Keras dispone de diferentes redes pre-entrenadas. En esta tutorial vamos a usar la red convolucional Inception Resnet V2 

En esta parte del código se realiza la carga de los pesos de la red pre-entrenada mediante la función InceptionResNetV2() y luego se realiza un procesamiento de los datos de entrada. 

Es importante destacar el parámetro de la funcion InceptionResNetV2: “include_top=False”. Este parámetro le indica a la red que no debe incluir la última capa de la red, destinada a realizar la predicción final. Esta capa está preparada para realizar las predicciones de todas las clases de ImageNet. Como nosotros queremos predecir únicamente nuestras 5 clases, queremos sustituir esta capa por una capa personalizada. Nuestra red ya no dispone de la última capa, por lo que vamos a añadirle una capa “softmax” de 5 clases. Esto va a permitir que la red haga la predicción de las clases que nos interesan y las entrenadas de ImageNet. 


In [ ]:
from keras.applications import InceptionResNetV2
from keras.applications import VGG16
from keras.applications import InceptionV3
from keras import models
from keras import layers
from keras.layers import Dropout
from keras.layers import AveragePooling2D, GlobalAveragePooling2D
from keras.models import Model

# Forma secuencial 

Vamos a usar como modelo base el modelo de la red InceptionResNetV2

In [ ]:
# base 
conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
#conv_base = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
conv_base.summary()

In [ ]:
# cabeza
model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(layers.Dense(num_classes, activation='softmax'))
model.summary()

# Forma Functional API

In [ ]:
# base
#base_Modelo = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=input_shape)

In [ ]:
# cabeza
#head_Modelo = base_Modelo.output
#head_Modelo = GlobalAveragePooling2D()(head_Modelo)
#head_Modelo = Dense(256, activation="relu")(head_Modelo)
#head_Modelo = Dense(num_classes, activation="softmax")(head_Modelo)

#model_api = Model(inputs=base_Modelo.input, outputs=head_Modelo)
#model_api.summary()

In [ ]:
# congelando las capas de la CNN, la cnn base.

print('número de capas entrenables de conv base:', len(model.trainable_weights))
conv_base.trainable = False
print('número de capas entrenables de conv base: ''después de congelar capas de conv base:', len(model.trainable_weights))
model.summary()

# Compilar Modelos

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
#model_api.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Ajustar el modelo

In [ ]:
batch_size = 20
epochs = 20

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()


score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))

# Data Augmentation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

#train_datagen = ImageDataGenerator() #rescale=1./255  (la imàgen ya está entre -1 y 1)
train_datagen = ImageDataGenerator(rotation_range=45, width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, zoom_range=0.2, horizontal_flip=True, fill_mode='nearest')

val_datagen = ImageDataGenerator()

batch_size = 20
epochs = 20

train_generator = train_datagen.flow(x_train, y_train , batch_size=batch_size)
validation_generator = val_datagen.flow(x_val, y_val , batch_size=batch_size)

In [ ]:
index = np.random.choice(len(train_generator))
img_path = train_generator[index]
img = img_path[0]
x = img[0,:,:,:]
normalizar_imagen(x)

In [ ]:
print("Valores mínimos y máximos de la imagen")
maxi=(np.max(x))
mini=(np.min(x))
print(mini,maxi )

In [ ]:
# parámetros del generador
steps_per_epoch=x_train.shape[0] // batch_size       
validation_steps=x_val.shape[0] // batch_size  

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, validation_data=validation_generator, validation_steps=validation_steps)

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))

In [ ]:
test_datagen = ImageDataGenerator()
test_steps=x_test.shape[0] // batch_size  
test_generator = test_datagen.flow(x_test, y_test , batch_size=batch_size)
test_loss, test_acc = model.evaluate_generator(test_generator, steps=test_steps)
print('test loss:', test_loss)
print('test acc:', test_acc)
print("CNN Error: %.2f%%" % (100-test_acc*100))

#  Congelar las capas

El siguiente paso consiste en decidir que capas congelar. Tendremos que hacer varias pruebas hasta encontrar un número de capas a congelar adecuado. Para tomar la decisión de si un modelo es apropiado o no, consultaremos los datos de validación, no los de test. Los datos de test deben usarse solo al final para ver si el modelo funciona. Tomar decisiones de diseño en base a los resultados en el conjunto de test es un error grave, pues nos lleva a construir una solución que solo funciona con nuestros datos.

A continuación vemos como congelar capas (trainable=False). Como nuestro modelo posee 782 capas, vamos a ir modificando este parámetro para ver los porcentajes de clasificación de la red.

Otro punto importante es el tiempo dedicado al entrenamiento. Como se puede ver, a medida que se reduce el número de capas congeladas, el tiempo de entrenamiento crece. Por lo que es muy interesante conseguir una red que funcione con muchas capas congeladas. Si se congelan pocas o ninguna ninguna capa, el tiempo de entrenamiento de cada epoch se dispara. Y si lo que se pretende es entrenar la red desde cero, ya es necesario acceder a grandes recursos (Clusters, GPUs, etc).

In [ ]:
LAYERS_TO_FREEZE=700
for layer in model.layers[:LAYERS_TO_FREEZE]:
    layer.trainable = True
    
model.compile(optimizer="adam",loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
batch_size = 20
epochs = 20

history = model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=1, validation_data=(x_val, y_val))

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochss = range(1, len(acc) + 1)
plt.plot(epochss, acc, 'r--o', label='Training acc')
plt.plot(epochss, val_acc, 'b--x', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochss, loss, 'r--o', label='Training loss')
plt.plot(epochss, val_loss, 'b--x', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])
print("CNN Error: %.2f%%" % (100-score[1]*100))


# Predicciones

In [ ]:
import numpy as np
index = np.random.choice(list(range(len(x_test))), 1)[0]
im = x_test[index]
target_class = {0:"Daisy",1:"Dandelion",2:"Roses",3:"Sunflowers",4:"Tulips"}
#target_class = {0:"Esteban",1:"Gabriel",2:"Gonzalo",3:"Héctor",4:"Seba"}

print('la imagen de test:')
normalizar_imagen(im)

print('la clase que predice la red es: ', target_class[np.argmax(model.predict(np.reshape(im, [1,img_rows, img_cols,3])), -1)[0]])

In [ ]:
import numpy as np
# predicciones de nuestra CNN con los datos de test
predicted_classes = model.predict_classes(x_test)

# revisemos cuales predicciones son correctas e incorrectas
correct_indices = np.nonzero(predicted_classes == YY_test)[0]
incorrect_indices = np.nonzero(predicted_classes != YY_test)[0]


plt.figure(figsize=(10,6))
for i, correct in enumerate(correct_indices[:9]):
    plt.subplot(3,3,i+1)
    normalizar_imagen(x_test[correct])
    plt.title("Predicted {}, Class {}".format(predicted_classes[correct], YY_test[correct]))

plt.figure(figsize=(10,6))
for i, incorrect in enumerate(incorrect_indices[:9]):
    plt.subplot(3,3,i+1)
    normalizar_imagen(x_test[incorrect])
    plt.title("Predicted {}, Class {}".format(predicted_classes[incorrect], YY_test[incorrect]))

# Matriz de confusión y reportes

In [ ]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score, precision_score, recall_score 

# classification report
print('Reporte de Clasificación:')
print(classification_report( YY_test,predicted_classes))

# confusion matrix 
cm = confusion_matrix(YY_test,predicted_classes)

# mostrar los resultados
print('Matriz de confusión:')
print(cm)

# Print f1, precision, and recall scores
print('Precision:')
print(precision_score(YY_test, predicted_classes , average="macro"))
print('Recall:')
print(recall_score(YY_test, predicted_classes , average="macro"))
print('F1:')
print(f1_score(YY_test, predicted_classes, average="macro"))